In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

In [ ]:
from PIL import Image
from pathlib import Path, PurePath
import re
import numpy as np
import decimal

In [ ]:
def import_lores_img_set(img_set_path, n_leds, img_prefix='IMG_',
                       img_format='TIF'):
    """Imports the files of the Lo-Res image set and returns all of them
    as 3-dimentional arrays (numpy.ndarray). In Fourier Ptychogaphy,
    the Lo-Res set is the result of illuminating a sample under a
    microscope with different incident angles.
    
    Args:
        img_set_path: (string) Route of the Lo-Res input image set.
        n_leds: (int) Number of LEDs using during the photo capture
            procces.
        img_prefix: (string) Common prefix of the Lo-Res photos'
            filenames.
        img_format: (string) File format of the Lo-Res photos.
    
    Returns:
        An array (numpy.ndarray) of imported images as array objects.
    """
    p = Path(img_set_path)
    str_n_leds = str(n_leds)
    re_wide_num_range = '[0-' + str_n_leds[0] + ']'
    for i in range(len(str_n_leds) - 1):
         re_wide_num_range += '[0-9]'
    # Creates a list with the files within p (Path) whose name begins
    # with glob_pattern (string). glob() does not support regexp.
    glob_pattern = img_prefix + re_wide_num_range + '.*'
    file_list = list(p.glob(glob_pattern))
    file_list.sort()  # Sorts the file list.
    # Create a regexp and use it to filter file_list (list).
    str_regex = img_prefix + '(?!0+\.)'
    if len(str_n_leds) > 1:
        str_regex += '(?!' + str_n_leds[0]
        for i in range(1, len(str_n_leds)):
            str_regex += '[' + str(int(str_n_leds[i]) + 1) + '-9]'
        str_regex += ')'
    str_regex += re_wide_num_range
    str_regex += '.('+ img_format.lower() + '|'
    str_regex += img_format.upper() + ')'
    filtered_file_list = list()
    img_cube = list()
    # Convert regexp pattern to a regexp object.
    regexp = re.compile(str_regex)
    # Store the images as 2-dimensionals arrays within a 3-dimensional
    # array.
    for item in file_list:
        m = regexp.search(str(item))
        if m and str(item).endswith(m.group(0)):
            img_path = str(item)
            img_num = m.group(0).split('_')[1].split('.')[0]
            print('<FP-ImageRead-' + img_num + '>')
            img = Image.open(img_path)
            arr_img = np.array(img)
            # If img.close() is before the conversion, arr_img
            # (numpy.ndarray) will contain TiffImageFile elements.
            img.close()
            img_cube.append(arr_img)
    return np.array(img_cube)

In [ ]:
def round_half_up(num):
    """In Python 3, the round() function had changed from Python 2. For
    example, in Python 3, round(2.5) returns 2 (int) such round(1.5).
    For that case, it is possible to get 3 (int) instead 2 using the
    code below [1].
    
    [1] Barthelemy. (2014). Python 3.x rounding behavior. Message
    posted to
    http://stackoverflow.com/questions/10825926/python-3-x-rounding-behavior
    
    Args:
        num: (float) Number to round.
    Returns:
        The rounded number.
    """
    return int(decimal.Decimal(num).quantize(decimal.Decimal(1),
                                             rounding=decimal.ROUND_HALF_UP))

In [ ]:
def merge_imgs(img_path_lst, mode='rgb', name='IMG_RESULT'):
    """Merges images into a new one using PIL.Image.merge() and stores
    it in the same path of the first source file with the same parent
    path. The route of each image is specified as an item of a list of
    strings.
    
    Args:
        img_path: (list) List of source image paths as strings.
        mode: (string) The mode for the output image supported by PIL.
            See https://pillow.readthedocs.io/en/3.1.x/handbook/concepts.html#concept-modes
        filename: (string) Filename of the output image. If it exists,
            it will change to <name's value>_<number (from 1)>.

    Returns:
        The path of the output image as a string.
    """
    img_src = list(map(lambda path: Image.open(path), img_path_lst))
    merger = Image.merge(mode.upper(), tuple(img_src))
    merger_parent = Path(img_path_lst[0]).parent
    merger_filename = name + Path(img_path_lst[0]).suffix 
    merger_path = PurePath(Path(img_path_lst[0]).parent,
                           merger_filename)
    c = 1
    while Path(merger_path).exists():
        merger_filename = (name + '_' + str(c) +
                           Path(img_path_lst[0]).suffix)
        merger_path = PurePath(Path(img_path_lst[0]).parent,
                               merger_filename)
        c = c + 1
    merger_path_str = merger_path.as_posix()
    print('<ImgMerge>')
    merger.save(merger_path_str)
    print(merger_path_str)
    # Close the opened image file.
    map(lambda pil_img: pil_img.close(), img_src)
    return merger_path_str